<a href="https://colab.research.google.com/github/2403a52227-hue/-NLP/blob/main/Assignment_8_2(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import string
from collections import Counter
from nltk.util import ngrams
import math
from collections import defaultdict
import nltk
nltk.download('punkt')
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
d1="data science is fun"
d2="machine learning is powerful"
d3="natural language processing is interesting"
d4="deep learning uses neural networks"
d5="python is popular for ai"
d6="ai is transforming industries"
d7="models learn from data"
d8="statistics helps in analysis"
d9="big data requires computation"
d10="algorithms solve problems"

In [ ]:
# Combine all documents
dataset = [d1, d2, d3, d4, d5, d6, d7, d8, d9, d10]
text = " ".join(dataset)
text = text.lower()
import re, string
text = re.sub(f"[{string.punctuation}]", "", text)
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text)
print("Tokens:\n", tokens)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Tokens:
 ['data', 'science', 'is', 'fun', 'machine', 'learning', 'is', 'powerful', 'natural', 'language', 'processing', 'is', 'interesting', 'deep', 'learning', 'uses', 'neural', 'networks', 'python', 'is', 'popular', 'for', 'ai', 'ai', 'is', 'transforming', 'industries', 'models', 'learn', 'from', 'data', 'statistics', 'helps', 'in', 'analysis', 'big', 'data', 'requires', 'computation', 'algorithms', 'solve', 'problems']


In [ ]:
from collections import Counter
word_counts = Counter(tokens)
rare_words = {word for word, count in word_counts.items() if count == 1}
print("Rare Words:\n", rare_words)
tokens_unk = [word if word not in rare_words else "UNK" for word in tokens]
print("\nTokens After UNK:\n", tokens_unk)

Rare Words:
 {'algorithms', 'from', 'statistics', 'transforming', 'uses', 'neural', 'science', 'networks', 'popular', 'natural', 'analysis', 'computation', 'fun', 'python', 'powerful', 'big', 'helps', 'for', 'interesting', 'solve', 'language', 'models', 'processing', 'machine', 'learn', 'deep', 'industries', 'in', 'requires', 'problems'}

Tokens After UNK:
 ['data', 'UNK', 'is', 'UNK', 'UNK', 'learning', 'is', 'UNK', 'UNK', 'UNK', 'UNK', 'is', 'UNK', 'UNK', 'learning', 'UNK', 'UNK', 'UNK', 'UNK', 'is', 'UNK', 'UNK', 'ai', 'ai', 'is', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'data', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'data', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK']


**N-GRAM MODEL**

In [ ]:
from nltk.util import ngrams

def build_ngram(tokens, n):
    return Counter(ngrams(tokens, n))
# Without UNK
unigram = build_ngram(tokens, 1)
bigram = build_ngram(tokens, 2)
trigram = build_ngram(tokens, 3)
# With UNK
unigram_unk = build_ngram(tokens_unk, 1)
bigram_unk = build_ngram(tokens_unk, 2)
trigram_unk = build_ngram(tokens_unk, 3)
print("Sample Unigram:\n", list(unigram.items())[:5])
print("Sample Bigram:\n", list(bigram.items())[:5])
print("Sample Trigram:\n", list(trigram.items())[:5])


Sample Unigram:
 [(('data',), 3), (('science',), 1), (('is',), 5), (('fun',), 1), (('machine',), 1)]
Sample Bigram:
 [(('data', 'science'), 1), (('science', 'is'), 1), (('is', 'fun'), 1), (('fun', 'machine'), 1), (('machine', 'learning'), 1)]
Sample Trigram:
 [(('data', 'science', 'is'), 1), (('science', 'is', 'fun'), 1), (('is', 'fun', 'machine'), 1), (('fun', 'machine', 'learning'), 1), (('machine', 'learning', 'is'), 1)]


In [ ]:
import math

def unigram_probability(model, word, total_words, vocab_size, k=0):
    count = model.get((word,), 0)
    return (count + k) / (total_words + k * vocab_size)


In [ ]:
def perplexity(tokens, model, vocab_size, k=0):
    total_words = sum(model.values())
    N = len(tokens)
    log_prob = 0

    for word in tokens:
        prob = unigram_probability(model, word, total_words, vocab_size, k)
        log_prob += math.log2(prob)

    return 2 ** (-log_prob / N)


**predicting probability for next words**

In [ ]:
# Unigram P(w)
def unigram_prob(w, unigram_counts, V, k=1):
    total = sum(unigram_counts.values())
    count = unigram_counts.get((w,), 0)
    return (count + k) / (total + k*V)
# Bigram P(w2 | w1)
def bigram_prob(w1, w2, bigram_counts, unigram_counts, V, k=1):
    count_w1_w2 = bigram_counts.get((w1, w2), 0)
    count_w1 = unigram_counts.get((w1,), 0)
    return (count_w1_w2 + k) / (count_w1 + k*V)
# Trigram P(w3 | w1, w2)
def trigram_prob(w1, w2, w3, trigram_counts, bigram_counts, V, k=1):
    count_w1_w2_w3 = trigram_counts.get((w1, w2, w3), 0)
    count_w1_w2 = bigram_counts.get((w1, w2), 0)
    return (count_w1_w2_w3 + k) / (count_w1_w2 + k*V)


In [ ]:
# Unigram prediction
def predict_next_word_unigram(unigram_counts, V):
    # Return word with maximum probability
    probs = {w[0]: unigram_prob(w[0], unigram_counts, V) for w in unigram_counts}
    return max(probs, key=probs.get)
# Bigram prediction
def predict_next_word_bigram(prev_word, bigram_counts, unigram_counts, V):
    probs = {w: bigram_prob(prev_word, w, bigram_counts, unigram_counts, V) for w in vocab}
    return max(probs, key=probs.get)
# Trigram prediction
def predict_next_word_trigram(prev_word1, prev_word2, trigram_counts, bigram_counts, V):
    probs = {w: trigram_prob(prev_word1, prev_word2, w, trigram_counts, bigram_counts, V) for w in vocab}
    return max(probs, key=probs.get)


In [ ]:
# Define V (vocabulary size)
V = len(unigram)
# Define vocab (set of unique words)
vocab = {word[0] for word in unigram.keys()}
next_word_uni = predict_next_word_unigram(unigram, V)
print("Next word prediction (Unigram):", next_word_uni)
# Bigram
next_word_bi = predict_next_word_bigram("machine", bigram, unigram, V)
print("Next word prediction (Bigram) after 'machine':", next_word_bi)

# Trigram
next_word_tri = predict_next_word_trigram("machine", "learning", trigram, bigram, V)
print("Next word prediction (Trigram) after 'machine learning':", next_word_tri)

Next word prediction (Unigram): is
Next word prediction (Bigram) after 'machine': learning
Next word prediction (Trigram) after 'machine learning': is
